In [10]:
class Keyboard:
    
    def front(self, letter):       # converts letter to signal
        signal = "ABCDEFGHIJKLMNOPQRSTUVWXYZ".find(letter)
        return signal
    def back(self, signal):
        letter="ABCDEFGHIJKLMNOPQRSTUVWXYZ"[signal]
        return letter

In [11]:
class Plugboard:
    def __init__(self,pairs):
        self.left="ABCDEFGHIJKLMNOPQRSTUVWXYZ"
        self.right="ABCDEFGHIJKLMNOPQRSTUVWXYZ"
        for x in pairs:
            A=x[0]
            B=x[1]
            A_loc=self.left.find(A)
            B_loc=self.left.find(B)
            self.left=self.left[:A_loc] + B + self.left[A_loc+1:]     # swap for plugboard
            self.left=self.left[:B_loc] + A + self.left[B_loc+1:]
            
    def front(self,signal):
        letter=self.right[signal]
        signal=self.left.find(letter)
        return signal
    
    def back(self,signal):
        letter=self.left[signal]
        signal=self.right.find(letter)
        return signal

In [12]:
class Rotor:
    def __init__(self, wiring, notch):
        self.left="ABCDEFGHIJKLMNOPQRSTUVWXYZ"
        self.right=wiring
        self.notch=notch
        
    def front(self,signal):
        letter=self.right[signal]
        signal=self.left.find(letter)
        return signal
    
    def back(self,signal):
        letter=self.left[signal]
        signal=self.right.find(letter)
        return signal
    
    def show(self):              
        print(self.left)
        print(self.right)
        print("")
        
    
    def rotate(self,n=1, front=True):# rings part no need front=true/else
        for i in range(n):
            if front:
                self.left=self.left[1:]+self.left[0]
                self.right=self.right[1:]+self.right[0]
            else:
                self.left=self.left[25]+self.left[:25]
                self.right=self.right[25]+self.right[:25]
        
    def rotate_to_letter(self,letter):
        n = "ABCDEFGHIJKLMNOPQRSTUVWXYZ".find(letter)
        self.rotate(n)
    
    def set_ring(self,n):          # ring part
        # rotate the rotor back
            self.rotate(n-1, front=False)
        # adjust the turnover notch in relationship to the wiring
            n_notch="ABCDEFGHIJKLMNOPQRSTUVWXYZ".find(self.notch)
            self.notch="ABCDEFGHIJKLMNOPQRSTUVWXYZ"[(n_notch-n)%26]
            
        



In [13]:
class Reflector:
    def __init__(self, wiring):
        self.left="ABCDEFGHIJKLMNOPQRSTUVWXYZ"
        self.right=wiring
        
    def reflect(self,signal):
        letter=self.right[signal]
        signal=self.left.find(letter)
        return signal

In [14]:
class Enigma:                                  
    def __init__(self,re,Rotor1,Rotor2,Rotor3,pb,kb):
        self.re=re
        self.Rotor1=Rotor1
        self.Rotor2=Rotor2
        self.Rotor3=Rotor3
        self.pb=pb
        self.kb=kb
    def set_key(self,key):
        self.Rotor1.rotate_to_letter(key[0])
        self.Rotor2.rotate_to_letter(key[1])
        self.Rotor3.rotate_to_letter(key[2])
        
    def set_rings(self, rings):             
        self.Rotor1.set_ring(rings[0])
        self.Rotor2.set_ring(rings[1])
        self.Rotor3.set_ring(rings[2])
        
    def encode(self,letter):
        if self.Rotor2.left[0]==self.Rotor2.notch and self.Rotor3.left[0]==self.Rotor3.notch:
            self.Rotor1.rotate()
            self.Rotor2.rotate()
            self.Rotor3.rotate()
        elif self.Rotor2.left[0]==self.Rotor2.notch: # double step anomaly
            self.Rotor1.rotate()
            self.Rotor2.rotate()
            self.Rotor3.rotate()
            
        elif self.Rotor3.left[0]==self.Rotor3.notch:
            self.Rotor2.rotate()
            self.Rotor3.rotate()
        else:
            self.Rotor3.rotate()
             
        signal=self.kb.front(letter)
        signal=self.pb.front(signal)
        signal=self.Rotor3.front(signal)
        signal=self.Rotor2.front(signal)
        signal=self.Rotor1.front(signal)
        signal=self.re.reflect(signal)
        signal=self.Rotor1.back(signal)
        signal=self.Rotor2.back(signal)
        signal=self.Rotor3.back(signal)
        signal=self.pb.back(signal)
        letter=self.kb.back(signal)
        return letter

In [17]:
I=Rotor("EKMFLGDQVZNTOWYHXUSPAIBRCJ","Q")          # taken from Wikipedia
II=Rotor("AJDKSIRUXBLHWTMCQGZNPYFVOE","E")
III=Rotor("BDFHJLCPRTXVZNYEIWGAKMUSQO","V")
IV=Rotor("ESOVPZJAYQUIRHXLNFTGKDCMWB","J")
V=Rotor("VZBRGITYUPSDNHLXAWMJQOFECK","Z")
A=Reflector("EJMZALYXVBWFCRQUONTSPIKHGD")
B=Reflector("YRUHQSLDPXNGOKMIEBFZCWVJAT")
C=Reflector("FVPJIAOYEDRZXWGCTKUQSBNMHL")

KB=Keyboard()
PB=Plugboard(["AS","DF","GH","JK"])


ENIGMA=Enigma(A,I,II,III,PB,KB)

ENIGMA.set_rings((1,3,5))

Input=input("enter string to be decoded:")
Key_encode=input("enter key for enigma:")

ENIGMA.set_key(Key_encode)  

Output=""
for x in Input:
    Output=Output + ENIGMA.encode(x)
print(Output)

enter string to be decoded:TRIAL
enter key for enigma:ZZZ
HGRMO


In [18]:
# cracking the Enigma basic approach
# working

input1=input("enter string to be decoded:")

import time
h1=time.time()

alphabet="ABCDEFGHIJKLMNOPQRSTUVWXYZ"

# finding out which key is the correct one

for x in range(0,26):
    for y in range(0,26):
        for z in range(0,26):
            I=Rotor("EKMFLGDQVZNTOWYHXUSPAIBRCJ","Q")          # taken from Wikipedia
            II=Rotor("AJDKSIRUXBLHWTMCQGZNPYFVOE","E")
            III=Rotor("BDFHJLCPRTXVZNYEIWGAKMUSQO","V")
            IV=Rotor("ESOVPZJAYQUIRHXLNFTGKDCMWB","J")
            V=Rotor("VZBRGITYUPSDNHLXAWMJQOFECK","Z")
            A=Reflector("EJMZALYXVBWFCRQUONTSPIKHGD")
            B=Reflector("YRUHQSLDPXNGOKMIEBFZCWVJAT")          # re-initializing enigma to ground state each iteration
            C=Reflector("FVPJIAOYEDRZXWGCTKUQSBNMHL")

            KB=Keyboard()
            PB=Plugboard(["AS","DF","GH","JK"])




            ENIGMA=Enigma(A,I,II,III,PB,KB)

            # set the rings
            ENIGMA.set_rings((1,3,5))
            key=(alphabet[x]+alphabet[y]+alphabet[z])
            ENIGMA.set_key(key)
            output=""
            for a in input1:
                output=output + ENIGMA.encode(a)
                if output==Input:
                    print("key is",key)
                    break
h2=time.time()
print("time taken is",h2-h1)
            

enter string to be decoded:HGRMO
key is ZZZ
time taken is 0.9983887672424316


In [9]:
# working but time is not correct

count=1
# cracking the Enigma basic approach
input1=input("enter string to be decoded:")
import time
h1=time.time()

alphabet="ABCDEFGHIJKLMNOPQRSTUVWXYZ"
# finding out which key is the correct one
for mirror in range(1,4):
    if mirror==1:
        mirror=A
    elif mirror==2:
        mirror=B
    else:
        mirror=C
    for x in range(0,26):
        for y in range(0,26):
            for z in range(0,26):
                I=Rotor("EKMFLGDQVZNTOWYHXUSPAIBRCJ","Q")          # taken from Wikipedia
                II=Rotor("AJDKSIRUXBLHWTMCQGZNPYFVOE","E")
                III=Rotor("BDFHJLCPRTXVZNYEIWGAKMUSQO","V")
                IV=Rotor("ESOVPZJAYQUIRHXLNFTGKDCMWB","J")
                V=Rotor("VZBRGITYUPSDNHLXAWMJQOFECK","Z")
                A=Reflector("EJMZALYXVBWFCRQUONTSPIKHGD")
                B=Reflector("YRUHQSLDPXNGOKMIEBFZCWVJAT")          # re-initializing enigma to ground state each iteration
                C=Reflector("FVPJIAOYEDRZXWGCTKUQSBNMHL")
                KB=Keyboard()
                PB=Plugboard(["AS","DF","GH","JK"])
                ENIGMA=Enigma(mirror,I,II,III,PB,KB)
                # set the rings
                ENIGMA.set_rings((1,3,5))
                key=(alphabet[x]+alphabet[y]+alphabet[z])
                ENIGMA.set_key(key)
                output=""
                for a in input1:
                    output=output + ENIGMA.encode(a)
                    if output==Input:
                        print("key is",key,"and reflector is",count)
                        break
    
    count+=1
h2=time.time()
print("time taken is",h2-h1)
            

enter string to be decoded:TRIAL
key is POG and reflector is 1
time taken is 3.036919593811035


In [79]:
# not working
I=Rotor("EKMFLGDQVZNTOWYHXUSPAIBRCJ","Q")          # taken from Wikipedia
II=Rotor("AJDKSIRUXBLHWTMCQGZNPYFVOE","E")
III=Rotor("BDFHJLCPRTXVZNYEIWGAKMUSQO","V")
IV=Rotor("ESOVPZJAYQUIRHXLNFTGKDCMWB","J")
V=Rotor("VZBRGITYUPSDNHLXAWMJQOFECK","Z")
A=Reflector("EJMZALYXVBWFCRQUONTSPIKHGD")
B=Reflector("YRUHQSLDPXNGOKMIEBFZCWVJAT")          # re-initializing enigma to ground state each iteration
C=Reflector("FVPJIAOYEDRZXWGCTKUQSBNMHL")

KB=Keyboard()
PB=Plugboard(["AS","DF","GH","JK"])




ENIGMA=Enigma(A,I,II,III,PB,KB)

# set the rings
ENIGMA.set_rings((1,3,5))



count=1
count2=1
# cracking the Enigma basic approach
input1=input("enter string to be decoded:")
import time
h1=time.time()

alphabet="ABCDEFGHIJKLMNOPQRSTUVWXYZ"
# finding out which key is the correct one
for rotor1 in range(1,6):
    if rotor1==1:
        rotor1=I
    elif rotor1==2:
        rotor1=II
    elif rotor1==3:
        rotor1=III
    elif rotor1==4:
        rotor1=IV
    else:
        rotor1=V
    for mirror in range(1,4):
        if mirror==1:
            mirror=A
            if output==Input:
                break
        elif mirror==2:
            mirror=B
            if output==Input:
                break
        else:
            mirror=C
        for x in range(0,26):
            for y in range(0,26):
                for z in range(0,26):
                    I=Rotor("EKMFLGDQVZNTOWYHXUSPAIBRCJ","Q")          # taken from Wikipedia
                    II=Rotor("AJDKSIRUXBLHWTMCQGZNPYFVOE","E")
                    III=Rotor("BDFHJLCPRTXVZNYEIWGAKMUSQO","V")
                    IV=Rotor("ESOVPZJAYQUIRHXLNFTGKDCMWB","J")
                    V=Rotor("VZBRGITYUPSDNHLXAWMJQOFECK","Z")
                    A=Reflector("EJMZALYXVBWFCRQUONTSPIKHGD")
                    B=Reflector("YRUHQSLDPXNGOKMIEBFZCWVJAT")          # re-initializing enigma to ground state each iteration
                    C=Reflector("FVPJIAOYEDRZXWGCTKUQSBNMHL")
                    KB=Keyboard()
                    PB=Plugboard(["AS","DF","GH","JK"])
                    ENIGMA=Enigma(mirror,rotor1,II,III,PB,KB)
                    # set the rings
                    ENIGMA.set_rings((1,3,5))
                    key=(alphabet[x]+alphabet[y]+alphabet[z])
                    ENIGMA.set_key(key)
                    output=""
                    for a in input1:
                        output=output + ENIGMA.encode(a)
                        if output==Input:
                            print("key is",key,"and reflector is",count%3,"and rotor 1 is",count2)
                            break

            count+=1
        count2+=1
h2=time.time()
print("time taken is",h2-h1)

enter string to be decoded:BDRWB
key is BOG and reflector is 2 and rotor 1 is 1
key is NCG and reflector is 0 and rotor 1 is 9
time taken is 40.24186968803406
